In [1]:
import sys

sys.path.append('/home/rustambaku13/Documents/Warwick/flink-streaming-gnn/python')

In [2]:
import jax
from flax.linen import relu, softmax, sigmoid, Module
from nn.jax.multi_layer_dense import MultiLayerDense
import jax.numpy as jnp
import pandas as pd
import numpy as np

## Data Preprocessing


In [5]:
edges = pd.read_csv("../python/dataset/cora/edges.csv", header=None, names=["src", "dest"] )
vertices = pd.read_csv("../python/dataset/cora/group-edges.csv", header=None, names=['vertex', "label"])

unique_features = vertices.label.unique()
one_hot_features = jnp.eye(unique_features.shape[0])
label_feature_map = {k:one_hot_features[i] for i,k in enumerate(unique_features)}
vertices['feature'] = vertices.label.map(lambda x: label_feature_map[x])
vertices['train'] = np.random.rand(vertices.shape[0]) < 0.2
vertices['feature_train'] = vertices.label.map(lambda x: label_feature_map[x])
vertex_index_map = {vertex: index for (index,vertex) in enumerate(vertices.vertex.values)}

edge_list = jnp.vstack(edges.apply(lambda x: [vertex_index_map[x.src], vertex_index_map[x.dest]], axis=1))

In [105]:
vertices['feature'] && vertices['train']

SyntaxError: invalid syntax (<ipython-input-105-cb73e4df1045>, line 1)

In [103]:
jnp.vstack(list(vertices.feature.values)) & vertices.train.values

ValueError: Incompatible shapes for broadcasting: ((2708, 7), (1, 2708))

In [29]:
count_edges = edges.groupby("dest").count()

In [34]:
count_edges[count_edges.src==5]

,src
dest,
164,5
434,5
910,5
3231,5
5069,5
...,...
1153577,5
1153943,5
1154123,5


In [35]:
edges[edges.dest==434]

,src,dest
159,10798,434
259,114,434
3443,4649,434
4148,6213,434
5166,8703,434


## Model Creation

In [ ]:
class GNNLayer(Module):
    message: Module
    update: Module
    agg: str
    
    def __call__(self, vertex_features, edges):
        
        